In [17]:
!pip install torchaudio



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms.functional import to_tensor
import matplotlib.pyplot as plt
from tqdm import tqdm



In [2]:
from networks.vision_transformer import SwinUnet


c:\Users\tasni\miniconda3\envs\tf\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tasni\miniconda3\envs\tf\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
class Config:
    root_path = "./Dataset"  # <-- This must exist!
    img_size = 224
    num_classes = 3
    base_lr = 0.01
    batch_size = 4
    max_epochs = 100
    n_gpu = 1
    num_workers = 4
    eval_interval = 5
    seed = 42
    snapshot_path = "./swin_output"
    pretrained_ckpt = "./pretrained_ckpt/swin_tiny_patch4_window7_224.pth"

args = Config()


# Create output directory if it doesn't exist
os.makedirs(args.snapshot_path, exist_ok=True)

# Set random seed for reproducibility
torch.manual_seed(args.seed)


In [4]:
from PIL import Image

class HepaticDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.image_files = sorted(os.listdir(image_dir))

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        mask_path = os.path.join(self.mask_dir, self.image_files[idx])

        image = Image.open(img_path).convert("L")
        mask = Image.open(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return {
            'image': image.float(),
            'label': mask.long().squeeze()
        }


In [5]:
train_transform = transforms.Compose([
    transforms.Resize((args.img_size, args.img_size)),
    transforms.ToTensor(),
])

train_dataset = HepaticDataset(
    image_dir=os.path.join(args.root_path, '2D_Sliced_Images'),
    mask_dir=os.path.join(args.root_path, '2D_Sliced_Masks'),
    transform=train_transform
)

train_loader = DataLoader(train_dataset, batch_size=args.batch_size, shuffle=True, num_workers=args.num_workers)


In [6]:
class DiceLoss(nn.Module):
    def __init__(self, num_classes):
        super(DiceLoss, self).__init__()
        self.num_classes = num_classes

    def forward(self, input, target, smooth=1e-5, softmax=True):
        if softmax:
            input = torch.softmax(input, dim=1)

        target_onehot = torch.eye(self.num_classes)[target].permute(0, 3, 1, 2).to(input.device)

        dims = (0, 2, 3)
        intersection = torch.sum(input * target_onehot, dims)
        cardinality = torch.sum(input + target_onehot, dims)

        dice = (2. * intersection + smooth) / (cardinality + smooth)
        return 1. - dice.mean()


In [11]:
pip install yacs


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from yacs.config import CfgNode as CN
import yaml

# Load YAML into a dictionary
with open('configs/swin_tiny_patch4_window7_224_lite.yaml', 'r') as f:
    yaml_cfg = yaml.safe_load(f)

# Convert dictionary to CfgNode (nested access support)
config = CN(yaml_cfg)


In [28]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [32]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
print("CUDA available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0))

CUDA available: False


AssertionError: Torch not compiled with CUDA enabled

In [31]:
print(torch.backends.cudnn.enabled)

True


In [33]:
pip uninstall torch torchvision torchaudio -y


Found existing installation: torch 2.7.1
Uninstalling torch-2.7.1:
  Successfully uninstalled torch-2.7.1
Found existing installation: torchvision 0.22.1
Uninstalling torchvision-0.22.1:
  Successfully uninstalled torchvision-0.22.1
Note: you may need to restart the kernel to use updated packages.


You can safely remove it manually.
You can safely remove it manually.


In [34]:
pip cache purge

Files removed: 1118 (3445.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (27 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.22.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.7.1%2Bcu118-cp39-cp39-win_amd64.whl.metadata (6.8 kB)
   ---------------------------------------- 0.0/2.8 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.8 GB 10.1 MB/s eta 0:04:40
   ---------------------------------------- 0.0/2.8 GB 11.4 MB/s eta 0:04:07
   ---------------------------------------- 0.0/2.8 GB 11.5 MB/s eta 0:04:05
   ---------------------------------------- 0.0/2.8 GB 11.9 MB/s eta 0:03:57
   ---------------------------------------- 0.0/2.8 GB 12.2 MB/s eta 0:03:51
   ---------------------------------------- 0.0/2.8 GB 12.5 MB/s eta 0:03:44
   ---------------------------------------- 0.0/2.8

ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\tasni\miniconda3\envs\tf\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\tasni\miniconda3\envs\tf\lib\site-packages\pip\_vendor\urllib3\response.py", line 561, in read
    data = self._fp_read(amt) if not fp_closed else b""
  File "C:\Users\tasni\miniconda3\envs\tf\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in _fp_read
    data = self._fp.read(chunk_amt)
  File "C:\Users\tasni\miniconda3\envs\tf\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 98, in read
    data: bytes = self.__fp.read(amt)
  File "C:\Users\tasni\miniconda3\envs\tf\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\tasni\miniconda3\envs\tf\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\tasni\miniconda3\envs\tf\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  Fi

In [3]:
nvcc --version

NameError: name 'nvcc' is not defined

In [4]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [5]:
print("CUDA Version:", torch.version.cuda)

NameError: name 'torch' is not defined

from yacs.config import CfgNode as CN
import torch
import os
from networks.vision_transformer import SwinUnet

# ---- Load YAML config ----
from yaml import safe_load

with open('configs/swin_tiny_patch4_window7_224_lite.yaml', 'r') as f:
    yaml_cfg = safe_load(f)

config = CN(yaml_cfg)

# ---- Set extra training args ----
class Args:
    root_path = "./Dataset"
    img_size = config.DATA.IMG_SIZE
    num_classes = 3
    base_lr = 0.01
    batch_size = 4
    max_epochs = 100
    n_gpu = 1
    num_workers = 4
    eval_interval = 5
    seed = 42
    snapshot_path = "./swin_output"

args = Args()

# ---- Ensure output folder exists ----
os.makedirs(args.snapshot_path, exist_ok=True)
torch.manual_seed(args.seed)

# ---- Step 1: Create the model ----
model = SwinUnet(
    config=config,
    img_size=args.img_size,
    num_classes=args.num_classes,
    zero_head=True
)

# ---- Step 2: Load pretrained weights from config path ----
model.load_from(config)

# ---- Step 3: Wrap in DataParallel if multiple GPUs ----
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)

# ---- Step 4: Move to GPU ----
model = model.cuda()


In [ ]:
pip install addict


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import yaml
from addict import Dict

# Load YAML file
with open('configs/swin_tiny_patch4_window7_224_lite.yaml', 'r') as f:
    config_dict = yaml.safe_load(f)

# Convert to dot-accessible format
config = Dict(config_dict)


In [ ]:
print(args.img_size)

224


In [ ]:
def to_2tuple(x):
    return x if isinstance(x, tuple) else (x, x)

In [ ]:
 tupImagesize=to_2tuple(args.img_size)

In [ ]:
# Initialize the model
model = SwinUnet(config=config,
                 img_size=tupImagesize,
                 num_classes=args.num_classes,
                 zero_head=True)

# Load pretrained checkpoint
# model.load_from(config)

# Use DataParallel if multiple GPUs
if args.n_gpu > 1:
    model = nn.DataParallel(model)

# Move to GPU
model = model.cuda()


SwinTransformerSys expand initial----depths:[2, 2, 2, 2];depths_decoder:[1, 2, 2, 2];drop_path_rate:0.2;num_classes:3


IndexError: tuple index out of range